# Ejercicio 1

## Creación de una bases de datos de películas extraídas de un API

En este ejercicio vamos a poner en práctica lo que hemos aprendido sobre la extracción de contenido de APIs y la creación de una base de datos con sus tablas. Siguiendo las fases propuestas, conseguiremos extraer datos de películas, almacenarlos en un DataFrame y finalmente crear una base de datos con la estructura adecuada.

Fase 1: Extracción de datos de películas

Nuestro primer objetivo es extraer los datos de películas de la API. En Adalab nos hemos creado un API
muy sencilla que contiene la siguiente información:
Datos a extraer:
- Título
- Año de lanzamiento
- Duración (en minutos)
- Género
- Contenido para adultos (sí o no)


El objetivo es extraer 100 películas de esta API utilizando el siguiente endpoint:
https://beta.adalab.es/resources/apis/pelis/pelis.json

In [1]:
import requests
import pandas as pd
import numpy as np
import mysql.connector
from mysql.connector import Error

In [2]:
url_films = "https://beta.adalab.es/resources/apis/pelis/pelis.json"
films = requests.get(url_films)

In [3]:
films.status_code

200

In [4]:
film_data = films.json()
film_data

[{'id': 1,
  'titulo': 'The Godfather',
  'año': 1972,
  'duracion': 175,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 2,
  'titulo': 'The Godfather Part II',
  'año': 1974,
  'duracion': 202,
  'genero': 'Crimen',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 3,
  'titulo': 'Pulp Fiction',
  'año': 1994,
  'duracion': 154,
  'genero': 'Crimen',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 4,
  'titulo': 'Forrest Gump',
  'año': 1994,
  'duracion': 142,
  'genero': 'Drama',
  'adultos': False,
  'subtitulos': ['es', 'en', 'fr']},
 {'id': 5,
  'titulo': 'The Dark Knight',
  'año': 2008,
  'duracion': 152,
  'genero': 'Acción',
  'adultos': False,
  'subtitulos': ['es', 'en']},
 {'id': 6,
  'titulo': 'Fight Club',
  'año': 1999,
  'duracion': 139,
  'genero': 'Drama',
  'adultos': True,
  'subtitulos': ['es', 'en']},
 {'id': 7,
  'titulo': 'Inception',
  'año': 2010,
  'duracion': 148,
  'genero': 'Ciencia ficción',
  'adultos

In [5]:
df_film_data = pd.DataFrame(film_data)
df_film_data

,id,titulo,año,duracion,genero,adultos,subtitulos
0,1,The Godfather,1972,175,Crimen,False,"[es, en]"
1,2,The Godfather Part II,1974,202,Crimen,False,"[es, en]"
2,3,Pulp Fiction,1994,154,Crimen,True,"[es, en]"
3,4,Forrest Gump,1994,142,Drama,False,"[es, en, fr]"
4,5,The Dark Knight,2008,152,Acción,False,"[es, en]"
...,...,...,...,...,...,...,...
95,96,La vita è bella,1997,116,Drama,False,"[es, en, it]"
96,97,Requiem for a Dream,2000,102,Drama,True,"[es, en]"
97,98,Memento,2000,113,Thriller,True,"[es, en]"
98,99,Eternal Sunshine of the Spotless Mind,2004,108,Drama,False,"[es, en]"


# Inserción de los Datos en la Base de Datos

En este apartado vamos a realizar la inserción de los datos extraídos en las tablas creadas en nuestra base de datos MySQL.

In [7]:
try:
    cnx = mysql.connector.connect(
        host='localhost',
        user='root',
        password='AlumnaAdalab',
    )
    print('Conexión exitosa')
except Error as e:
    print('Error al conectar:', e)

try:
    mycursor = cnx.cursor()
    query = "CREATE DATABASE IF NOT EXISTS pelis_db"
    mycursor.execute(query)
    print("Query exitosa")
except:
    print("Error.")

Conexión exitosa
Query exitosa


In [8]:
mycursor.execute("USE pelis_db")

query = '''CREATE TABLE films(
	ID_film INT,
    title VARCHAR(70) NOT NULL,
    release_date YEAR,
    runtime INT,
    genre VARCHAR(45),
    rating_18 BOOLEAN,
    PRIMARY KEY (id_film)
);'''
mycursor.execute(query)

In [9]:
mycursor.execute("USE pelis_db")
query = '''CREATE TABLE languages(
    ID_language INT NOT NULL,
    language VARCHAR(45) UNIQUE NOT NULL,
    PRIMARY KEY (ID_language)
);'''
mycursor.execute(query)

In [10]:
mycursor.execute("USE pelis_db")

query = '''CREATE TABLE subtitles(
    ID_film INT NOT NULL,
    ID_language INT NOT NULL,
    PRIMARY KEY (ID_film, ID_language),
    FOREIGN KEY (ID_film) REFERENCES films(ID_film),
    FOREIGN KEY (ID_language) REFERENCES languages(ID_language)
);'''

mycursor.execute(query)

In [ ]:
# Si se ponen las 3 juntas fallan, sin embargo por separado funcionan correctamente

mycursor.execute("USE pelis_db")

query = '''CREATE TABLE films(
	ID_film INT,
    title VARCHAR(70) NOT NULL,
    release_date DATE
    runtime INT,
    genre VARCHAR(45),
    rating_18 BOOLEAN,
    subtitles INT
    PRIMARY KEY (id_film)
);'''

query = '''CREATE TABLE languages(
    ID_language INT NOT NULL,
    language VARCHAR(45) UNIQUE NOT NULL,
    PRIMARY KEY (ID_language)
);'''

query = '''CREATE TABLE subtitles_films(
    ID_film INT NOT NULL,
    ID_language INT NOT NULL,
    PRIMARY KEY (ID_film, ID_language),
    FOREIGN KEY (ID_film) REFERENCES films(ID_film),
    FOREIGN KEY (ID_language) REFERENCES subt_language(ID_language)
);'''

mycursor.execute(query)

In [14]:
mycursor.execute("USE pelis_db")
query_insert = """INSERT INTO films (id, titulo, año, duracion, genero, 
                              adultos, subtitulos) 
VALUES (%s, %s, %s, %s, %s, %s, %s)
"""

df_clean = df_film_data.replace({np.nan: None, 'nan': None, 'NaN': None})
data = df_clean.tolist()
mycursor.executemany(query_insert, data)
cnx.commit()

AttributeError: 'DataFrame' object has no attribute 'tolist'